<a href="https://colab.research.google.com/github/HongYue1/LightNovel-Crawler-Colab/blob/main/lightnovel_crawler.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 <font color="#3399ff" size=10><b> How to use:</b></font>
### 1. Click on  <font color="yellow" size=5><b> connect </b></font>on the top right and wait untill it finishs connecting.
### 2. Before running a block, make sure to <font color="red" size=5> <b> fill the required data first. </b></font>
### 3. Run each block, wait for the block to finish before running the next block, the block finishes when you see a green check mark<font color ="#1bc757" size=5><b> ✔</b></font> next to the run button.
###4. After uploading finish you will be given the links as an output.
### 5. Note: to download several novels you don't need to redo all these steps, just download a novel then upload it, then go back to the download section and download the next novel then upload it....etc , <font color="red" size=5> <b> you can only upload one novel a time.</br></font>
### <font color="#00c3e3" size=6><b> Look at the buttom of this notebook for A list of tested and working sources.</b></font>
# <font color="FF1493" size=5> <b> Enjoy <3 </b> </font>
# <a href="https://github.com/HongYue1/LightNovel-Crawler-Colab">Github</a> <br >


## ⏬ Install LightNovel-crawler

In [ ]:
# @markdown <font size =5 color="yellow"> ⬅️ Click Here to install</font>
from google.colab import output
!pip install -U lightnovel-crawler
!pip install gofilepy-api

output.clear()
print("Done!")

## ⏬ Download Novel

In [ ]:
#@markdown <font size =5 color="yellow">⬅️ Click Here to Download</font>

#@markdown <h4> Enter the Link:

Link = ""#@param {type:"string"}
#@markdown <center><h3><font color="#3399ff"><b>⚙️ Advanced Options ⚙️</b></font></h3></center>

#@markdown <h4> Enter the range of the chapters separted by a space or enter <font color="#3399ff" size=5><b>All</b></font>  to download all chapters.
#@markdown <h5> Example to download chapters between 1 and 100 : 1 100
Range = "All"#@param {type:"string"}


Format="epub"

def download():

  Misc_command=" -f --suppress"


  if(Range=="All"):
    !lncrawl --single -o "/content/novels" -s $Link --format $Format --all $Misc_command
  else:
    !lncrawl --single -o "/content/novels" -s $Link --format $Format --range $Range $Misc_command

download()

##<img src='https://raw.githubusercontent.com/biplobsd/OneClickRun/master/img/zip.png' height="30" alt="netdata"> Archive and Upload

In [ ]:
# @markdown <font size =5 color="yellow"> ⬅️ Upload</font>

import os
import json
import requests
import threading
import urllib.parse
import subprocess
from gofilepy import GofileClient


def get_file_path():
    """Get the first file path from the specified directory."""
    directory_path = "/content/novels/epub/"
    try:
        file_list = os.listdir(directory_path)
        if not file_list:
            raise FileNotFoundError("No files found in the directory")

        first_file = file_list[0]
        file_name_without_extension = os.path.splitext(first_file)[0]
        filename = file_name_without_extension + ".epub"
        return directory_path + first_file, filename
    except Exception as e:
        print(f"Error getting file path: {e}")
        return None, None


def upload_to_gofile(file_path):
    """Upload file to Gofile.io."""
    try:
        client = GofileClient()
        with open(file_path, "rb") as file_obj:
            file = client.upload(file=file_obj)
        return file.page_link
    except Exception as e:
        # print(f"Gofile upload error: {e}")
        return "Upload failed"


def upload_to_filesvc(file_path):
    """Upload file to files.vc."""
    try:
        with open(file_path, "rb") as file_obj:
            files = {"file": file_obj}
            headers = {"X-Account-ID": "9215854213114742"}
            response = requests.post(
                "https://api.files.vc/upload", files=files, headers=headers, timeout=10
            )
            response.raise_for_status()
            return response.json()["file_url"]
    except Exception as e:
        # print(f"Files.vc upload error: {e}")
        return "Upload failed"


def upload_to_buzzheavier(file_path, filename):
    """Upload file to buzzheavier.com."""
    try:
        buzzheavier_url = "https://w.buzzheavier.com/" + urllib.parse.quote(
            filename, safe=""
        )
        result = subprocess.run(
            ["curl", "-#o-", "-T", file_path, buzzheavier_url],
            capture_output=True,
            text=True,
            check=True,
            timeout=10,
        )
        json_data = json.loads(result.stdout)
        file_id = json_data["data"]["id"]
        return "https://buzzheavier.com/" + file_id
    except Exception as e:
        # print(f"Buzzheavier upload error: {e}")
        return "Upload failed"


def main():
    """Main function to coordinate uploads."""

    # Get file path
    file_path, filename = get_file_path()
    if not file_path:
        print("Could not find file to upload")
        return

    print(f"Uploading file: {filename}")

    # Concurrent uploads
    gofile_result = {"url": "Upload in progress..."}

    # Start Gofile upload in a thread
    def gofile_thread_func():
        gofile_result["url"] = upload_to_gofile(file_path)

    gofile_thread = threading.Thread(target=gofile_thread_func)
    gofile_thread.daemon = True
    gofile_thread.start()

    # Upload to other services while Gofile is uploading
    #filesvc_url = upload_to_filesvc(file_path)
    buzzheavier_url = upload_to_buzzheavier(file_path, filename)

    # Wait for Gofile thread with timeout
    gofile_thread.join(timeout=15)
    if gofile_thread.is_alive():
        gofile_result["url"] = "Upload timed out"

    # Display results
    print("\nUploads completed!")
    print("Links to the file: (choose any)")
    print(f"Gofile: {gofile_result['url']}")
    #print(f"Files.vc: {filesvc_url}")
    print(f"Buzzheavier: {buzzheavier_url}")


if __name__ == "__main__":
    main()


## List of Working Sources sorted by speed.
⭐ = Personal Recommedation  

<font color="red" size="4"> Note:</font> <font size="3">working doesn't mean that it has the novel you want or the novel isn't missing chapters in that source, so always take notice of the the number of chapters while downloading.

In [ ]:
#@markdown ## <font color="yellow" size=6> ⬅️ Fetch working sources

import pandas as pd
from urllib.parse import urlparse, parse_qs
from IPython.display import display, Markdown, HTML
import re


def fetch_google_sheet(sheet_url):
    """
    Fetch data from Google Sheets URL and return as a DataFrame.

    Args:
        sheet_url (str): URL to Google Sheets document

    Returns:
        pandas.DataFrame: Data from the sheet
    """
    try:
        # Parse URL components
        parsed_url = urlparse(sheet_url)

        # Extract document ID from URL path
        path_parts = parsed_url.path.split("/")
        if len(path_parts) < 4:
            raise ValueError("Invalid Google Sheets URL format")
        doc_id = path_parts[3]

        # Extract gid from URL query parameters or fragment
        params = parse_qs(parsed_url.query)
        fragment = parse_qs(parsed_url.fragment)

        # Look for gid in both query params and fragment
        gid = params.get("gid", ["0"])[0]
        if gid == "0" and "gid" in fragment:
            gid = fragment.get("gid", ["0"])[0]

        # Create CSV export URL
        export_url = f"https://docs.google.com/spreadsheets/d/{doc_id}/export?format=csv&gid={gid}"

        # Read data from Google Sheets
        return pd.read_csv(export_url)

    except Exception as e:
        print(f"Error fetching Google Sheet: {e}")
        return pd.DataFrame()  # Return empty DataFrame on error


def format_cell_content(value):
    """Format cell content for better display in HTML table."""
    if pd.isna(value):
        return ""

    # Convert to string
    value_str = str(value)

    # Format URLs as clickable links with target="_blank" to open in new tab
    url_pattern = r"https?://\S+"
    if re.match(url_pattern, value_str):
        # For Colab, using explicit HTML anchor tags instead of markdown links
        return f'<a href="{value_str}" target="_blank">{value_str}</a>'

    # Truncate long text
    if len(value_str) > 50:
        return value_str[:47] + "..."

    return value_str


def display_as_enhanced_html_table(df, max_rows=None):
    """
    Display DataFrame as a styled HTML table with enhanced formatting.

    Args:
        df (pandas.DataFrame): Data to display
        max_rows (int, optional): Maximum number of rows to display
    """
    if df.empty:
        display(HTML("<p><em>No data available</em></p>"))
        return

    # Limit rows if specified
    if max_rows and len(df) > max_rows:
        display_df = df.head(max_rows)
        footer_note = f"<p><em>Showing {max_rows} of {len(df)} rows</em></p>"
    else:
        display_df = df
        footer_note = ""

    # Generate styled HTML table for better control and compatibility with Colab
    html = "<style>\n"
    html += "table {border-collapse: collapse; width: 80%; margin: 15px 0;}\n"
    html += "th {background-color: #89b4fa; font-size: 18px; color: black; text-align: center; padding: 12px; white-space: nowrap; font-weight: bold;}\n"
    html += "td {background-color: #313244; font-size: 16px; border: 1px solid #45475a; padding: 8px; text-align: center;}\n"
    html += "</style>\n"

    html += "<table border='1'>\n"

    # Add header row
    html += "<tr>\n"
    for col in display_df.columns:
        html += f"<th>{col}</th>\n"
    html += "</tr>\n"

    # Add data rows with custom formatting for each cell
    for _, row in display_df.iterrows():
        html += "<tr>\n"
        for value in row:
            formatted_value = format_cell_content(value)
            html += f"<td>{formatted_value}</td>\n"
        html += "</tr>\n"

    html += "</table>\n"

    # Add footer note if needed
    if footer_note:
        html += footer_note

    # Display as HTML
    display(HTML(html))


def detect_and_format_numeric_columns(df):
    """Detect numeric columns and format them appropriately."""
    for col in df.columns:
        # Check if column contains numeric data
        if pd.api.types.is_numeric_dtype(df[col]):
            # Format integers without decimal places
            if df[col].apply(lambda x: x == int(x) if not pd.isna(x) else True).all():
                df[col] = (
                    df[col].fillna("").astype(str).replace(r"\.0$", "", regex=True)
                )
            # Format floats with 2 decimal places
            else:
                df[col] = df[col].apply(lambda x: f"{x:.2f}" if not pd.isna(x) else "")
    return df


def main():
    # Google Sheet URL
    input_url = "https://docs.google.com/spreadsheets/d/1SGfpvLFq7DzssklY90cmTKqQEBNeOBcAj1zSHDv-gfA/edit?usp=sharing"

    # Fetch data
    df = fetch_google_sheet(input_url)

    # Pre-process data for better display
    df = detect_and_format_numeric_columns(df)

    # Display the enhanced table
    display_as_enhanced_html_table(df, max_rows=500)


if __name__ == "__main__":
    main()
